Import opencl modules

In [1]:
from microsim.opencl.ramp.run import run_headless
from microsim.opencl.ramp.snapshot_convertor import SnapshotConvertor
from microsim.opencl.ramp.snapshot import Snapshot
from microsim.opencl.ramp.params import Params
from microsim.opencl.ramp.simulator import Simulator

Load starting snapshot

In [2]:
snapshot_filepath = "./opencl/snapshots/cache.npz"
snapshot = Snapshot.load_full_snapshot(path=snapshot_filepath)

Setup params

In [3]:
params = Params(
        retail_multiplier=1.0,
        primary_school_multiplier=1.0,
        secondary_school_multiplier=1.0,
        home_multiplier=1.0,
        work_multiplier=1.0,
        current_risk_beta=0.0165
    )

snapshot.params = params

Run OpenCL simulation for multiple repetitions

In [5]:
iterations = 100
repetitions = 5
summaries = []
num_seed_days = 5
use_gpu=False

for i in range(repetitions):
    # set the random seed of the model for each repetition, otherwise it is completely deterministic
    snapshot.seed_prngs(i)

    # seed initial infections using GAM initial cases
    snapshot.seed_initial_infections(num_seed_days=num_seed_days, data_dir="./opencl/data/")

    # Create a simulator and upload the snapshot data to the OpenCL device
    simulator = Simulator(snapshot, kernel_dir="./opencl/ramp/kernels/", gpu=use_gpu)
    simulator.upload_all(snapshot.buffers)
    
    summary, final_state = run_headless(simulator, snapshot, iterations, quiet=True)
    summaries.append(summary)

Running simulation: 100%|██████████| 100/100 [00:07<00:00, 13.28it/s]


Plot output data

In [ ]:
summary = summaries[0]

